# RAG Langchain PDF Example - Council Financial Plans
This example loads one or more PDF documents, splits the contents into chunks, loads these into a vector store, then uses a retriever to 
ask natural langauge questions.

This uses langchain, a popular package to chain all these opeartions together, and to use an underlying Generative AI model, in this case OpenAI.

In [65]:
# pip install openai,  pypdf,  python-dotenv,  langchain,  langchain-community,  langchain-openai,  langchain-text-splitters, chromadb

In [ ]:
import os

# Load the .env file.  This allows us to use environment variables in the .env file
from dotenv import load_dotenv
load_dotenv() # load the .env file

In [67]:
import textwrap

These are the imports that are required for the langchain to work

In [68]:
from langchain import hub
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


Get the PDF files(s) that we will want to analyse.

In [ ]:
pdf_dir = os.path.abspath("./pdf/")
pdf_dir

Load all PDFs in the directory

In [ ]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

directory_loader = DirectoryLoader(pdf_dir, glob="**/*.pdf", loader_cls=PyPDFLoader)
documents = directory_loader.load()
documents[:3]

In [ ]:
my_list = [1,2,3,'hello', 'world']

[x if isinstance(x, int) else 1 for x in my_list ]

In [ ]:
#type(documents),
#len(documents[10].page_content)

[len(document.page_content) for document in documents]

In [ ]:
print(textwrap.fill(documents[1].page_content, width = 80))

Use the ChatOpenAI class to create a language model

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")
llm

Split the text into smaller chunks based on sentences or characters

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)
chunks[:3]

In [ ]:
print(chunks[0].page_content)
print(chunks[1].page_content)
print(chunks[2].page_content)



In [ ]:
len(chunks)

In [ ]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(documents)
vectorstore = Chroma.from_documents(documents=chunks, embedding=OpenAIEmbeddings())
vectorstore

Create the RAG chain

In [ ]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
print(prompt.messages[0])

In [ ]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
rag_chain

In [81]:
#  Some example prompts
test_prompt1 = "Summarise Middlesborough council financial plan in two paragraphs?"
test_prompt2 = "In what ways are the financial plans of Middlesborough council and Derbyshire Dales similar?"

In [ ]:
response = rag_chain.invoke(test_prompt2)
wrapped_response = textwrap.fill(response, width=120)
print(wrapped_response)